# 保存模型(基于单隐层神经网络的手写数字识别)

# 载入数据

In [1]:
import tensorflow as tf

# 导入Tensorflow提供的读取MNIST的模块
import tensorflow.examples.tutorials.mnist.input_data as input_data

# 读取MNIST数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#定义全连接层函数
def fcn_layer(inputs,
              input_dim,
              output_dim,
              activation=None):
    
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1)) #以截断正态分布的随机数初始化W
    
    b = tf.Variable(tf.zeros([output_dim])) # 以0初始化b
    
    XWb = tf.matmul(inputs, W) + b #建立表达式: inputs * W + b
    
    if activation is None: # 默认有使用激活函数
        outputs = XWb
    else: 
        outputs = activation(XWb) # 若传入激活函数，则用其对输出结果进行变换
    
    return outputs

# 构建输入层

In [3]:
x = tf.placeholder(tf.float32, [None, 784], name="X")

# 构建模型

In [4]:
H1_NN = 256 # 隐藏层神经元数量256

# 构建隐藏层

In [5]:
#隐藏层神经元数量256
h1 = fcn_layer(inputs=x,
               input_dim=784,
               output_dim=H1_NN,
               activation=tf.nn.relu)

# 构建输出层 

In [6]:
forward = fcn_layer(inputs=h1,
                    input_dim=H1_NN,
                    output_dim=10,
                    activation=None)
pred = tf.nn.softmax(forward)

# 训练模型

In [7]:
#定义标签数据占位符
y = tf.placeholder(tf.float32, [None, 10], name="Y")

# 定义损失函数

In [8]:
#交叉熵
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=forward, labels=y))

# 设置训练参数

In [9]:
train_epochs = 40
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
learning_rate = 0.01

#存储模型的粒度
save_step = 10

#创建保存模型文件的目录
import os
ckpt_dir = "./ckpt_dir/"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

每个固定间隔（save_step)在检查点保存一个模型（.ckpt文件），可便于随时将模型拿来预测。
通常观察前几次的预测效果就可以粗略估计出神经网络设计得是否合理。

# 选择优化器

In [10]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

# 定义准确率

In [11]:
correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 训练并存储模型

In [12]:
#声明完成所有变量后，调用tf.train.Saver
saver = tf.train.Saver()

#记录训练开始时间
from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer,feed_dict={x: xs, y: ys})
    #total_batch个批次训练完成后，使用验证数据计算误差与准确率
    loss, acc = sess.run([loss_function, accuracy],
                         feed_dict={x: mnist.validation.images,
                                    y: mnist.validation.labels})
    if(epoch+1) % display_step ==0:
        print("Train Epoch:","%02d" % (epoch+1),
              "Loss=","{:.9f}".format(loss), " Accuracy=","{:.4f}".format(acc))
    
    if (epoch+1) % save_step == 0:
        saver.save(sess, os.path.join(ckpt_dir,
                                  'mnist_h256_model_{:06d}.ckpt'.format(epoch+1))) # 存储模型
        print('minist_h256_model_{:06d}.ckpt saved'.format(epoch+1))

saver.save(sess, os.path.join(ckpt_dir, 'mnist_h256_model.ckpt'))
print("Model saved!")

#显示运行总时间
duration = time()-startTime
print("Train Finished takes:","{:.2f}".format(duration))

Train Epoch: 01 Loss= 0.130000323  Accuracy= 0.9618
Train Epoch: 02 Loss= 0.145687342  Accuracy= 0.9628
Train Epoch: 03 Loss= 0.143346682  Accuracy= 0.9624
Train Epoch: 04 Loss= 0.156009227  Accuracy= 0.9676
Train Epoch: 05 Loss= 0.154769927  Accuracy= 0.9644
Train Epoch: 06 Loss= 0.150880605  Accuracy= 0.9686
Train Epoch: 07 Loss= 0.221782222  Accuracy= 0.9600
Train Epoch: 08 Loss= 0.192608073  Accuracy= 0.9706
Train Epoch: 09 Loss= 0.215352193  Accuracy= 0.9632
Train Epoch: 10 Loss= 0.187212616  Accuracy= 0.9646
minist_h256_model_000010.ckpt saved
Train Epoch: 11 Loss= 0.220813796  Accuracy= 0.9708
Train Epoch: 12 Loss= 0.200191468  Accuracy= 0.9698
Train Epoch: 13 Loss= 0.246806055  Accuracy= 0.9668
Train Epoch: 14 Loss= 0.241324484  Accuracy= 0.9702
Train Epoch: 15 Loss= 0.234268799  Accuracy= 0.9694
Train Epoch: 16 Loss= 0.294493049  Accuracy= 0.9716
Train Epoch: 17 Loss= 0.252464831  Accuracy= 0.9736
Train Epoch: 18 Loss= 0.304699868  Accuracy= 0.9706
Train Epoch: 19 Loss= 0.3204

# 评估模型

In [13]:
accu_test = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
print("Test Accuracy:", accu_test)

Test Accuracy: 0.9711
